# SIADS 591-592 Milestone 1 Project - Data Acquisition and Processing

## Greenhouse Gas (GHG) Emissions from Upstream and Midstream US Oil and Gas Operations

### Dataset 1: Greenhouse Gas Emissions Data from EPA.gov
**Processing steps:**  
Read processing steps in the main project report [here](EmissionsProject-Report-MasterNotebook-V3.ipynb/#ProcessEmissionData)  
Output after processing the dataset will be saved in below CSV file(s):
1. **Emissions_aggregatedData.csv** - This file will contain Reporting year, Company, Gas type, Sector, and GHG Emission volume and raking based on 2018 emission volume
Below two files are not currently used, but will be used in future projects
2. **Emissions_AggWithSecRank.csv** - This file contains Reporting year, Company and Emission volume, and a rank by accumulated emissions from 2009 to 2018.
3. **ND_AggregatedEmissions.csv** - Emission volumes from the North Dakota region only. Columns include: Reporting year, company name and GHG emission volume

### Dataset 2: Emission Data from other Industrial Sectors from EPA.gov
**Processing steps:**  
Read processing steps in the main project report [here](EmissionsProject-Report-MasterNotebook-V3.ipynb/#ProcessOtherIndustryData)  
Output after processing the dataset will be saved in below CSV file(s):  
1. **Emissions_OtherIndustries.csv** - Emissions from Other industries - columns - Reporting Year, Industrial Sector and Emission volume

### Dataset 3: Crude Oil and Natural Gas Production volumes from EIA.gov
**Processing steps:**  
Read processing steps in the main project report [here](EmissionsProject-Report-MasterNotebook-V3.ipynb/#ProcessProductionData)  
Output after processing the dataset will be saved in below CSV file(s):  
1. **ProductionVsEmissionSplit.csv** - This file combines production volumes and Emission volumes into a single dataframe. Production and Emission numbers are split in Emission-sector and Production-Product types, Data present in Key-value types from 2011 onwards

### Software and Hardware dependencies:  
Code was compiled and tested on below hardware and software configuration:  
```python
CPython 3.7.6
IPython 7.14.0

pandas 1.0.3
cleanco 1.3

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 62 Stepping 4, GenuineIntel
CPU cores  : 32
interpreter: 64bit
```

In [2]:
%load_ext watermark
%watermark -v -m -p pandas,cleanco

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.7.6
IPython 7.14.0

pandas 1.0.3
cleanco 1.3

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 62 Stepping 4, GenuineIntel
CPU cores  : 32
interpreter: 64bit


In [2]:
# Import required python libraries
import pandas as pd
#cleanco library will be used to standardize the company names
from cleanco import cleanco
import re

In [3]:
#Define reusable functions
def standardCompName(inName):
    return cleanco(inName).clean_name().upper()

## Processing Emissions Dataset

###### Read the six Emissions data excel spreadsheets, parse them and save the processed data into a CSV file
<a id='ProcessEmissionsData'></a>

In [4]:
#Read emission excel spreadsheets and parse them into pandas dataframe
FileList=['Midstream-CH4.xls','Midstream-CO2.xls','Midstream-N2O.xls','Upstream-CH4.xls','Upstream-CO2.xls','Upstream-N2O.xls']
folder_path = "./"

total_size = 0
df_list=[]
for file in FileList:
    file_path=folder_path+file
    
    fileName = file.split('.')[0].split('-')
    sector = fileName[0]
    gas = fileName[1]
    
    df = pd.read_excel(file_path, sheet_name=None,skiprows=[0,1,2,3,4,5])
    for year in df:
        df_key = sector+"_"+gas+"_"+year
        df_name=df_key+"_df"
        df_name = df[year]
        df_name['GAS']=gas
        df_name['SECTOR']=sector
        df_list.append(df_name)

# combine (union) all these dataframes into a single dataframe       
df_fullset = pd.concat(df_list)
# Some of the facilities are operated under partnership, we need to allocate emission quantities to individual parent companies
df_fullset['PARENT COMPANIES']=df_fullset['PARENT COMPANIES'].str.split(';')
df_fullset=df_fullset.explode('PARENT COMPANIES')

#Seperate the company name and contribution
regex=r'(?P<PARENT_COMPANY>[-\w\s\d,&./()#]+)([\(])(?P<CONTRIBUTION>[\(\d.]+)([%\)]*)'
#Compile the regular expression for better performance
compName_RE = re.compile(regex)

df_fullset[['PARENT_COMPANY','CONTRIBUTION_PERCENT']]=df_fullset['PARENT COMPANIES'].str\
.extract(compName_RE)[['PARENT_COMPANY','CONTRIBUTION']]

#Turn those numbers into float values
df_fullset[['GHG QUANTITY (METRIC TONS CO2e)','CONTRIBUTION_PERCENT']]=df_fullset[['GHG QUANTITY (METRIC TONS CO2e)','CONTRIBUTION_PERCENT']].apply(pd.to_numeric)

df_fullset['GHG_CONTRIBUTION']=df_fullset['GHG QUANTITY (METRIC TONS CO2e)']*df_fullset['CONTRIBUTION_PERCENT']*0.01
df_fullset['PARENT_COMPANY']=df_fullset['PARENT_COMPANY'].str.strip()

#Standardize company names
df_NameLookup = pd.read_csv(folder_path+'CompanyName_Lookup.csv',sep='|')
df_NameLookup['PARENT_COMPANY']=df_NameLookup['PARENT_COMPANY'].str.strip()

df_fullset=df_fullset.merge(df_NameLookup,left_on='PARENT_COMPANY', right_on='PARENT_COMPANY', how='left')

## Remove the rows that doesnt have company name, these are excel column wise sums
df_fullset=df_fullset[~df_fullset['STANDARD_COMPANY_NAME'].isna()]
#replace raw company names with standardized company names
df_fullset['PARENT_COMPANY']=df_fullset['STANDARD_COMPANY_NAME'].str.strip()

#Aggregate the data by Parent company, sector, reporting year and Emission type
df_Agg=df_fullset.groupby(['REPORTING YEAR','PARENT_COMPANY','GAS','SECTOR'])['GHG_CONTRIBUTION']\
.agg('sum').reset_index().sort_values(['PARENT_COMPANY','REPORTING YEAR'])


#2018 upstream emission rank
df_Agg_upstream=df_Agg[(df_Agg['REPORTING YEAR']==2018) & (df_Agg['SECTOR']=='Upstream')].groupby(['PARENT_COMPANY']).sum().sort_values('GHG_CONTRIBUTION',ascending=False).reset_index().reset_index().rename(columns={'index':'2018_UPSTREAM_RANK'})
df_Agg_upstream['2018_UPSTREAM_RANK']=df_Agg_upstream['2018_UPSTREAM_RANK']+1
df_Agg_upstream=df_Agg_upstream.drop(['REPORTING YEAR','GHG_CONTRIBUTION'],axis=1)
#df_Agg_upstream.head(10)

#2018 midstream emission rank
df_Agg_midstream=df_Agg[(df_Agg['REPORTING YEAR']==2018) & (df_Agg['SECTOR']=='Midstream')].groupby(['PARENT_COMPANY']).sum().sort_values('GHG_CONTRIBUTION',ascending=False).reset_index().reset_index().rename(columns={'index':'2018_MIDSTREAM_RANK'})
df_Agg_midstream['2018_MIDSTREAM_RANK']=df_Agg_midstream['2018_MIDSTREAM_RANK']+1
df_Agg_midstream=df_Agg_midstream.drop(['REPORTING YEAR','GHG_CONTRIBUTION'],axis=1)
df_Agg_midstream.head(10)

#2018 combined emission rank
df_Agg_overall=df_Agg[(df_Agg['REPORTING YEAR']==2018)].groupby(['PARENT_COMPANY']).sum().sort_values('GHG_CONTRIBUTION',ascending=False).reset_index().reset_index().rename(columns={'index':'2018_OVERALL_RANK'})
df_Agg_overall['2018_OVERALL_RANK']=df_Agg_overall['2018_OVERALL_RANK']+1
df_Agg_overall=df_Agg_overall.drop(['REPORTING YEAR','GHG_CONTRIBUTION'],axis=1)
df_Agg_overall.head(10)

# Add ranks to Aggregated dataset
df_Agg_withRanks=pd.merge(df_Agg,df_Agg_upstream,on='PARENT_COMPANY',how='left')\
.merge(df_Agg_midstream,on='PARENT_COMPANY',how='left')\
.merge(df_Agg_overall,on='PARENT_COMPANY',how='left')\
.sort_values('2018_UPSTREAM_RANK')

#Fill uncalculated ranks to max rank
df_Agg_withRanks['2018_UPSTREAM_RANK']=df_Agg_withRanks['2018_UPSTREAM_RANK'].fillna(max(df_Agg_withRanks['2018_UPSTREAM_RANK'])+1)
df_Agg_withRanks['2018_MIDSTREAM_RANK']=df_Agg_withRanks['2018_MIDSTREAM_RANK'].fillna(max(df_Agg_withRanks['2018_MIDSTREAM_RANK'])+1)


#Save a copy of processed data to CSV file for visualization and analysis
df_Agg_withRanks.to_csv('Emissions_aggregatedData.csv',index=False,sep='|')

df_Agg_withRanks.head(5)

,REPORTING YEAR,PARENT_COMPANY,GAS,SECTOR,GHG_CONTRIBUTION,2018_UPSTREAM_RANK,2018_MIDSTREAM_RANK,2018_OVERALL_RANK
4240,2011,HILCORP ENERGY,CH4,Midstream,104.5002,1.0,30.0,8.0
4262,2014,HILCORP ENERGY,N2O,Midstream,432.6312,1.0,30.0,8.0
4261,2014,HILCORP ENERGY,CO2,Upstream,594898.0000,1.0,30.0,8.0
4260,2014,HILCORP ENERGY,CO2,Midstream,760000.5073,1.0,30.0,8.0
4259,2014,HILCORP ENERGY,CH4,Upstream,215286.0000,1.0,30.0,8.0


In [5]:
df_Agg_withRanks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10226 entries, 4240 to 10225
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   REPORTING YEAR       10226 non-null  int64  
 1   PARENT_COMPANY       10226 non-null  object 
 2   GAS                  10226 non-null  object 
 3   SECTOR               10226 non-null  object 
 4   GHG_CONTRIBUTION     10226 non-null  float64
 5   2018_UPSTREAM_RANK   10226 non-null  float64
 6   2018_MIDSTREAM_RANK  10226 non-null  float64
 7   2018_OVERALL_RANK    8242 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 719.0+ KB


## Processing the Emissons from other Industrial sectors
<a id='DataPrep_OtherIndustries'></a>

In [4]:
df_industry = pd.read_csv(folder_path+'IndustryWiseGHGEmissions.csv')
# Unpivot the Yearly data, put it in columner format
df_industry=pd.melt(df_industry, id_vars=['Industry Sector'], var_name='Year', value_name='Emission')
#Remove yearly totals
df_industry=df_industry[df_industry['Industry Sector']!='Total']
#Write it to a csv file
df_industry.to_csv('Emissions_OtherIndustries.csv',index=False,sep='|')

In [5]:
df_industry.sample(5)

,Industry Sector,Year,Emission
262,Other industrial categories,2016,102.451658
210,Fossil fuel combustion: carbon dioxide,2011,754.627887
251,Natural gas and petroleum systems,2015,270.548521
112,Other industrial categories,2001,120.708074
124,Coal mining,2002,71.143086


## Oil and Gas Production Data processing
<a id='DataPrep_ProdDataPrep'></a>

In [6]:
# Read Natural Gas production file
dict_gasProduction = pd.read_excel(folder_path+'NaturalGas Production.xls',sheet_name=[1,2], usecols=[0,1],skiprows=[0,1], names=['Date','Production kBOE'])
df_gasProduction = pd.concat(dict_gasProduction.values())
df_gasProduction=df_gasProduction.groupby('Date').sum()
df_gasProduction['Product']='Natural Gas'
df_gasProduction['Production kBOE']=df_gasProduction['Production kBOE']*1000/6000

df_gas_annual=df_gasProduction.groupby(['Product',pd.Grouper(freq="Y")]).sum().reset_index()
df_gas_annual=df_gas_annual[df_gas_annual['Date']<'2020-01-01']

#Pickup the year part
df_gas_annual.Date = df_gas_annual.Date.dt.year


# Crude oil production
df_CrudeProduction=pd.read_excel(folder_path+'CrudeOil Production.xls',sheet_name=1, usecols=[0,1],skiprows=[0,1], names=['Date','Production kBOE'])
df_CrudeProduction['Product']='Crude'
df_CrudeProduction['Production kBOE']=df_CrudeProduction['Production kBOE']
df_CrudeProduction=df_CrudeProduction.set_index('Date')

df_crude_annual=df_CrudeProduction.groupby(['Product',pd.Grouper(freq="Y")]).sum().reset_index()
df_crude_annual=df_crude_annual[df_crude_annual['Date']<'2020-01-01']
#Pickup the year
df_crude_annual.Date = df_crude_annual.Date.dt.year


#Put both Datasets together in columner format
df_annual_production = pd.concat([df_crude_annual,df_gas_annual])
#Generate file for SP analysis
df_annual_production.to_csv('Processed_AnnualProductionData.csv', sep='|')
df_annual_production.head(5)

,Product,Date,Production kBOE
0,Crude,1920,442929.0
1,Crude,1921,472183.0
2,Crude,1922,557531.0
3,Crude,1923,732407.0
4,Crude,1924,713940.0


## Join the Emissions and Production Data using Reporting Year
<a id='Join_Prod_n_Emission'></a>

##### Merge Emissions and Production data into a columnar dataframe and write it to a CSV file for analysis and visualizations

In [7]:
df_individula = df_annual_production[df_annual_production.Date>2010]

df_combined=df_annual_production[df_annual_production.Date>2010].groupby('Date').sum().reset_index()
df_combined['Product']='Combined Production'

df_ProdSplit=pd.concat([df_combined,df_individula]).rename(columns={'Date':'REPORTING_YEAR','Product':'Key','Production kBOE':'Value'})
df_ProdSplit=df_ProdSplit.replace('Crude','Crude Production').replace('Natural Gas','Natural Gas Production')


df_emission_BySec=df_Agg_withRanks[['REPORTING YEAR','SECTOR','GHG_CONTRIBUTION']].groupby(['REPORTING YEAR','SECTOR']).sum().reset_index()

df_emission_comb=df_emission_BySec.groupby('REPORTING YEAR').sum().reset_index()
df_emission_comb['SECTOR']='Combined Emission'
df_emiss_split=pd.concat([df_emission_BySec, df_emission_comb])
df_emiss_split=df_emiss_split.rename(columns={'REPORTING YEAR':'REPORTING_YEAR','SECTOR':'Key','GHG_CONTRIBUTION':'Value'})
df_emiss_split=df_emiss_split.replace('Midstream','Midstream Emission').replace('Upstream','Upstream Emission')

df_prodvsEmission=pd.concat([df_ProdSplit,df_emiss_split]).reset_index().drop('index',axis=1)
df_prodvsEmission.to_csv('ProductionVsEmissionSplit.csv',sep='|',index=False)
df_prodvsEmission.sample(10)

,REPORTING_YEAR,Value,Key
7,2018,1.028334e+07,Combined Production
0,2011,7.008380e+06,Combined Production
40,2017,9.959268e+07,Upstream Emission
34,2014,1.079649e+08,Upstream Emission
44,2012,1.851053e+08,Combined Emission
15,2017,3.413418e+06,Crude Production
32,2013,1.041574e+08,Upstream Emission
39,2017,1.586330e+08,Midstream Emission
26,2019,6.863423e+06,Natural Gas Production
5,2016,8.762416e+06,Combined Production


###### Extract North Dakota specific emissions - Will be used in a future project for a deeper analysis of state and company level study
<a id='ND_EmissionData'></a>

In [8]:
df_ND_Emissions=df_fullset[df_fullset['STATE']=='ND'].copy()
df_ND_Emissions['PARENT_COMPANY']=df_ND_Emissions['PARENT_COMPANY'].str.strip()

df_ND_Agg=df_ND_Emissions.groupby(['REPORTING YEAR','PARENT_COMPANY','GAS','SECTOR'])['GHG_CONTRIBUTION']\
.agg('sum').reset_index().sort_values(['PARENT_COMPANY','REPORTING YEAR'])

df_ND_Agg=df_ND_Agg.rename(columns={'PARENT_COMPANY':'COMPANY'})

df_ND_Agg.to_csv('ND_AggregatedEmissions.csv',sep='|')
df_ND_Agg.head(5)

,REPORTING YEAR,COMPANY,GAS,SECTOR,GHG_CONTRIBUTION
242,2017,1804,CH4,Midstream,3818.0
243,2017,1804,CO2,Midstream,19480.0
298,2018,1804,CH4,Midstream,8913.0
299,2018,1804,CO2,Midstream,59969.0
300,2018,1804,N2O,Midstream,24.0


##### Prepare a dataframe with Rank by total value in last 9 years - Will be used in future project

In [9]:
#Prepare a dataframe with Rank by total value in last 9 years
df_Agg_SecRank=df_Agg_withRanks[['PARENT_COMPANY','SECTOR','GHG_CONTRIBUTION']]\
.groupby(['PARENT_COMPANY','SECTOR'])\
.sum().reset_index().sort_values('GHG_CONTRIBUTION',ascending=False).reset_index().drop('index',axis=1).reset_index()\
.rename(columns={'index':'RANK'})

df_Agg_SecRank['RANK']=df_Agg_SecRank['RANK']+1

df_Agg_SecRank.to_csv('Emissions_AggWithSecRank.csv',index=False,sep='|')

df_Agg_SecRank.head(10)

,RANK,PARENT_COMPANY,SECTOR,GHG_CONTRIBUTION
0,1,EXXONMOBIL,Midstream,6.637022e+07
1,2,WILLIAMS,Midstream,6.569103e+07
2,3,KINDER MORGAN,Midstream,6.347846e+07
3,4,ENERGY TRANSFER PARTNERS,Midstream,6.258268e+07
4,5,CONOCOPHILLIPS,Upstream,5.486298e+07
5,6,ENTERPRISE PRODUCTS PARTNERS,Midstream,5.338663e+07
6,7,EXXONMOBIL,Upstream,4.123950e+07
7,8,CHESAPEAKE ENERGY,Upstream,3.319011e+07
8,9,EOG RESOURCES,Upstream,3.103791e+07
9,10,DCP MIDSTREAM,Midstream,2.895804e+07


In [10]:
df_Agg_withRanks[df_Agg_withRanks['PARENT_COMPANY'].str.contains('CONOCO')].head()

,REPORTING YEAR,PARENT_COMPANY,GAS,SECTOR,GHG_CONTRIBUTION,2018_UPSTREAM_RANK,2018_MIDSTREAM_RANK,2018_OVERALL_RANK
1936,2011,CONOCOPHILLIPS,N2O,Upstream,1.705577e+03,5.0,14.0,10.0
1964,2016,CONOCOPHILLIPS,CO2,Upstream,2.105699e+06,5.0,14.0,10.0
1965,2016,CONOCOPHILLIPS,N2O,Midstream,1.629980e+03,5.0,14.0,10.0
1966,2016,CONOCOPHILLIPS,N2O,Upstream,1.462837e+03,5.0,14.0,10.0
1967,2017,CONOCOPHILLIPS,CH4,Midstream,2.219486e+05,5.0,14.0,10.0
